In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install kaggle nbdev

     ---------------------------------------- 63.3/63.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 64.8/64.8 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 58.1/58.1 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 67.6/67.6 kB 3.6 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77731 sha256=77a7e6853568fc59f163844060e3bcedcb98579ef94ec396608075588e4b6c7a
  Stored in directory: c:\users\karde\appdata\local\pip\cache\wheels\1b\22\79\e73b3e988388377c1cda3385bfa20c1799be101c2e12bbe3dc
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName("exemplo bla bla").config("spark.driver.memory","4g").config("spark.executor.memory","4g").getOrCreate()

In [3]:
# Some general imports 

import os
import sys

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
# from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

In [4]:
def plotHistogram(df, xcol, huecol=None):
    sns.histplot(data=df, x=xcol, hue=huecol, multiple="stack")

In [5]:
def plot(df, xcol, ycol):
    sns.lineplot(data=df, x=xcol, y=ycol)

In [6]:
def plotBar(df, xcol, ycol, huecol=None):
    sns.barplot(data=df, x=xcol, y=ycol, hue=huecol)

In [7]:
def plotScatter(df, xcol, ycol, huecol=None):
    sns.scatterplot(data=df, x=xcol, y=ycol, hue=huecol)

In [8]:
def plotScatterMatrix(df, huecol=None):
    sns.pairplot(data=df, hue=huecol)

In [9]:
def plotCorrelationMatrix_1(df, annot=False):
    # compute the correlation matrix
    corr = df.corr()
    
    # generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    #cmap='coolwarm'

    # draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=annot,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [10]:
def plotCorrelationMatrix_2(df):
    # compute a correlation matrix and convert to long-form
    corr_mat = df.corr().stack().reset_index(name="correlation")
    # draw each cell as a scatter point with varying size and color
    g = sns.relplot(
        data=corr_mat,
        x="level_0", y="level_1", hue="correlation", size="correlation",
        palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
        height=10, sizes=(50, 250), size_norm=(-.2, .8),
    )

    # tweak the figure to finalize
    g.set(xlabel="", ylabel="", aspect="equal")
    g.despine(left=True, bottom=True)
    g.ax.margins(.02)
    for label in g.ax.get_xticklabels():
        label.set_rotation(90)
    for artist in g.legend.legendHandles:
        artist.set_edgecolor(".7")

In [11]:
# Some Spark related imports we will use hereafter

import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import row_number, monotonically_increasing_id,split
from pyspark.sql import Window


In [12]:
filename = "train.csv"
df_train = spark.read.csv(filename,header=True, sep=",", inferSchema=True)


In [13]:
filename = "test.csv"
df_teste = spark.read.csv(filename,header=True, sep=",", inferSchema=True)

In [14]:
filename = "train_labels.csv"
df_train_labs = spark.read.csv(filename,header=True, sep=",", inferSchema=True)

In [15]:
df_train.Schema() ####uaaaaaaaaaaaaiiiiiiiiii plllllaaaaaaaaaanccchhhhhhhhhhhhaaaaaaaaaaa


AttributeError: 'DataFrame' object has no attribute 'Schema'

In [16]:
df_teste.printSchema()

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- page: double (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable = true)
 |-- text: string (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- text_fqid: string (nullable = true)
 |-- fullscreen: integer (nullable = true)
 |-- hq: integer (nullable = true)
 |-- music: integer (nullable = true)
 |-- level_group: string (nullable = true)
 |-- session_level: integer (nullable = true)



In [17]:
df_teste2 = df_teste.withColumn(
    "id_new",
    row_number().over(Window.orderBy(monotonically_increasing_id()))-1
)

In [32]:
df_teste2.toPandas()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,indexkard
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,None,None,None,0-4,0
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,...,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,1
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,...,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,2
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,3
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723,20090312331414616,1001,1581679,map_hover,basic,22,NaN,NaN,NaN,NaN,...,484.0,None,tunic.wildlife,tunic.historicalsociety.entry,None,None,None,None,13-22,3723
3724,20090312331414616,1002,1583044,map_hover,basic,22,NaN,NaN,NaN,NaN,...,783.0,None,tunic.capitol_2,tunic.historicalsociety.entry,None,None,None,None,13-22,3724
3725,20090312331414616,1003,1583410,map_click,undefined,22,NaN,483.726363,-3.880047,456.0,...,NaN,None,tunic.capitol_2,tunic.historicalsociety.entry,None,None,None,None,13-22,3725
3726,20090312331414616,1004,1585841,navigate_click,undefined,22,NaN,192.372139,38.216178,383.0,...,NaN,None,chap4_finale_c,tunic.capitol_2.hall,None,None,None,None,13-22,3726


In [22]:
df_teste.schema
StructType(List(StructField(session_id,long,true),StructField(index,integer,true)))

NameError: name 'List' is not defined

In [16]:
df_teste.select("*").toPandas()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level
0,20090109393214576,0,0,cutscene_click,basic,0,NaN,-413.991405,75.685314,380.0,...,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,None,None,None,0-4,20090109393214576_0-4
1,20090109393214576,1,1965,person_click,basic,0,NaN,-105.991405,-63.314686,688.0,...,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,20090109393214576_0-4
2,20090109393214576,2,3614,person_click,basic,0,NaN,-418.991405,47.685314,375.0,...,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,20090109393214576_0-4
3,20090109393214576,3,5330,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,20090109393214576_0-4
4,20090109393214576,4,6397,person_click,basic,0,NaN,-110.991405,-57.314686,683.0,...,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,None,None,None,0-4,20090109393214576_0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3723,20090312331414616,1001,1581679,map_hover,basic,22,NaN,NaN,NaN,NaN,...,484.0,None,tunic.wildlife,tunic.historicalsociety.entry,None,None,None,None,13-22,20090312331414616_13-22
3724,20090312331414616,1002,1583044,map_hover,basic,22,NaN,NaN,NaN,NaN,...,783.0,None,tunic.capitol_2,tunic.historicalsociety.entry,None,None,None,None,13-22,20090312331414616_13-22
3725,20090312331414616,1003,1583410,map_click,undefined,22,NaN,483.726363,-3.880047,456.0,...,NaN,None,tunic.capitol_2,tunic.historicalsociety.entry,None,None,None,None,13-22,20090312331414616_13-22
3726,20090312331414616,1004,1585841,navigate_click,undefined,22,NaN,192.372139,38.216178,383.0,...,NaN,None,chap4_finale_c,tunic.capitol_2.hall,None,None,None,None,13-22,20090312331414616_13-22


In [19]:
df_train.printSchema()

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- page: double (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable = true)
 |-- text: string (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- text_fqid: string (nullable = true)
 |-- fullscreen: integer (nullable = true)
 |-- hq: integer (nullable = true)
 |-- music: integer (nullable = true)
 |-- level_group: string (nullable = true)



In [20]:
df_teste = df_teste.drop('session_level')

In [21]:
df_teste.printSchema()

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- page: double (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable = true)
 |-- text: string (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- text_fqid: string (nullable = true)
 |-- fullscreen: integer (nullable = true)
 |-- hq: integer (nullable = true)
 |-- music: integer (nullable = true)
 |-- level_group: string (nullable = true)



In [22]:
df_todo = df_train.union(df_teste)


In [23]:
df_todo = df_todo.withColumn(
    "id_new",
    row_number().over(Window.orderBy(monotonically_increasing_id()))-1
)

In [24]:
df_train_labs = df_train_labs.select(split("session_id", '_',-1).getItem(0).alias('session_id'),split("session_id", '_',-1).getItem(1).alias('question'),'correct')

In [25]:
df_train_labs = df_train_labs.withColumn("session_id",df_train_labs["session_id"].cast('float'))

In [26]:
df_train_labs.printSchema()

root
 |-- session_id: float (nullable = true)
 |-- question: string (nullable = true)
 |-- correct: integer (nullable = true)



In [34]:
df_todo.write.mode("overwrite").parquet("all_data_predict")

Py4JJavaError: An error occurred while calling o140.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j

In [28]:
df_train_labs.select("session_id").toPandas()

,session_id
0,2.009031e+16
1,2.009031e+16
2,2.009031e+16
3,2.009031e+16
4,2.009031e+16
...,...
424111,2.210022e+16
424112,2.210022e+16
424113,2.210022e+16
424114,2.210022e+16


In [ ]:
#colunios = split(df_train_labs.select('session_id'), '_')

In [113]:
listaasd = df_train_labs.collect()

In [29]:
df_todo.select("session_id").distinct().toPandas()

,session_id
0,20100114570801148
1,20100308381327090
2,20100016070249504
3,20100109380136844
4,20100112570372764
...,...
23560,22090614284270924
23561,22100221145014656
23562,20090312331414616
23563,20090312143683264


In [33]:
df_train_labs.select("session_id").distinct().toPandas()

,session_id
0,2.010022e+16
1,2.103012e+16
2,2.105061e+16
3,2.106001e+16
4,2.107022e+16
...,...
1430,2.201061e+16
1431,2.206021e+16
1432,2.207002e+16
1433,2.207060e+16


In [31]:
23565/18

1309.1666666666667

In [ ]:
listaasd

In [121]:
asd = df_todo.select("session_id").distinct().collect()

In [122]:
kar = [x for x in asd if asd in listaasd]

In [115]:
df_train_labs.select("session_id"==listaasd).show()

TypeError: Invalid argument, not a string or column: False of type <class 'bool'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [21]:
#df_todo = df_todo.select("*").withColumn("id_index", monotonically_increasing_id())

NameError: name 'monotonically_increasing_id' is not defined

In [ ]:
#df_todo.select("*").toPandas()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\anaconda\instalation\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\anaconda\instalation\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\anaconda\instalation\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\anaconda\instalation\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


In [19]:
df_teste.show(1, vertical=True)
teste_count = df_teste.count()
df_teste.show(1)

-RECORD 0------------------------------
 session_id     | 20090109393214576    
 index          | 0                    
 elapsed_time   | 0                    
 event_name     | cutscene_click       
 name           | basic                
 level          | 0                    
 page           | null                 
 room_coor_x    | -413.99140522263224  
 room_coor_y    | 75.6853138297722     
 screen_coor_x  | 380.0                
 screen_coor_y  | 259.0                
 hover_duration | null                 
 text           | undefined            
 fqid           | intro                
 room_fqid      | tunic.historicals... 
 text_fqid      | tunic.historicals... 
 fullscreen     | null                 
 hq             | null                 
 music          | null                 
 level_group    | 0-4                  
 session_level  | 20090109393214576... 
only showing top 1 row

+-----------------+-----+------------+--------------+-----+-----+----+-------------------+------

In [20]:
df_train.show(1, vertical=True)
train_count = df_train.count()
df_train.show(1)

-RECORD 0------------------------------
 session_id     | 20090312431273200    
 index          | 0                    
 elapsed_time   | 0                    
 event_name     | cutscene_click       
 name           | basic                
 level          | 0                    
 page           | null                 
 room_coor_x    | -413.99140522263224  
 room_coor_y    | -159.31468617022784  
 screen_coor_x  | 380.0                
 screen_coor_y  | 494.0                
 hover_duration | null                 
 text           | undefined            
 fqid           | intro                
 room_fqid      | tunic.historicals... 
 text_fqid      | tunic.historicals... 
 fullscreen     | null                 
 hq             | null                 
 music          | null                 
 level_group    | 0-4                  
only showing top 1 row

+-----------------+-----+------------+--------------+-----+-----+----+-------------------+-------------------+-------------+------------

In [ ]:
cols_to_forget = ['session_id']
cards_cols_interest = [x for x in df_teste.columns if x in cols_to_forget]

In [ ]:
df_teste.describe(cards_cols_interest).show()

In [ ]:
'session_id == 20100308381327090'

In [ ]:
df_aaa = ( df_teste
                .withColumn("session_id", F.when(F.col('session_id')=='20100308381327090', 1).otherwise(0))
            )
df_aaa

In [ ]:
df_teste.filter(F.expr('session_id == 20090312331414616')).toPandas()

In [ ]:
df_teste.where('session_id == 20100308381327090').show()

In [ ]:
#df_teste.groupBy('level').count().toPandas().sort(df_teste.col("level"),("count")).show(truncate=False)

In [ ]:
df_teste.groupBy('session_id').count().toPandas()


#incialmente pensamento em utilziar o dataset de teste para trabalhar com um dataset menor, foi verificado que no dataset de teste possui apenas 3 session ID unicos enquanto que no dataset de treino tem 11779 session id unicos por isso não podemos utilziar o dataset de teste, teremos de fazer uma reduçao apartir dp dataset de treino

In [ ]:
df_teste.groupBy('text').count().toPandas()

In [ ]:
df_teste.groupBy('event_name').count().toPandas()

In [ ]:
df_plot = df_teste.groupBy('event_name').count().toPandas()
plotBar(df_plot, 'event_name', 'count')
plt.title('grupos (event_name)')
plt.show()

In [ ]:
df_train[f.Column'session_id'].printSchema()
df_train.show(1, vertical=True)
train_count = df_train.count()
train_count
df_train.show(1)

In [ ]:
df_train_labs.show(5)

In [ ]:
print('\nNulls in all columns')
cols_to_forget = ['Credit Limit']
teste_cols_interest = [x for x in df_teste.columns]
for cl in teste_cols_interest:
    k = df_teste.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {teste_count} records ({k*100/teste_count:.2f}%)')

In [ ]:
test

In [ ]:
print('\nNulls in all columns')
cols_to_forget = ['fullscreen','hq','music','hover_duration']
train_cols_interest = [x for x in df_train.columns if df_train.columns not in cols_to_forget]
for cl in train_cols_interest:
    k = df_train.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {train_count} records ({k*100/train_count:.2f}%)')

In [ ]:
df_train.head(4)

In [ ]:
df_train.tail(4)

In [ ]:
train_cols_interest.remove('fullscreen')
train_cols_interest.remove('music')
train_cols_interest.remove('hq')
train_cols_interest.remove('page')
train_cols_interest.remove('hover_duration')
train_cols_interest.remove('text')
train_cols_interest.remove('text_fqid')

In [ ]:
for cl in train_cols_interest:
    k = df_train.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {train_count} records ({k*100/train_count:.2f}%)')

In [ ]:
teste_cols_interest.remove('fullscreen')
teste_cols_interest.remove('music')
teste_cols_interest.remove('hq')
teste_cols_interest.remove('page')

In [ ]:
for cl in teste_cols_interest:
    k = df_teste.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {teste_count} records ({k*100/teste_count:.2f}%)')

In [ ]:
print('\nUniqueness in teste:')
for cl in teste_cols_interest:
    k = df_teste.select(cl).distinct().count()
    print(f"Columns {cl} with {k} distinct values, out of {teste_count}"+f" records ({k*100/teste_count:.2f})%")

In [ ]:
print('\nUniqueness in treino:')
for cl in train_cols_interest:
    k = df_train.select(cl).distinct().count()
    print(f"Columns {cl} with {k} distinct values, out of {train_count}"+f" records ({k*100/train_count:.2f})%")

In [ ]:
#corr_matrix = Correlation.corr(df_teste, df_teste).collect()[0][0].toArray().tolist()

In [ ]:
#df_plot = pd.DataFrame(data = corr_matrix, index=df.teste.columns, columns=df.teste.columns)
#plotCorrelationMatrix_1(df_plot, annot=True)
#plt.title('Correlations among numerical features')
#plt.show()

In [ ]:
seed = 5
with_replacement = False
fraction = 0.2          # reduce to 20%
small_df_train = df_train.sample(withReplacement=with_replacement, 
                                               fraction=fraction, seed=seed)

In [ ]:
small_df_train.count()

In [ ]:
df_train.count()

In [80]:
small_df_train.write.mode("overwrite").parquet("small-df_train")

In [32]:
df_train_clean = spark.read.parquet('small-df_train')

In [33]:
df_train_clean.toPandas()

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/home/kardec/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kardec/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/kardec/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,21050410434308160,968,1726999,person_click,basic,20,NaN,-381.847136,110.712177,254.0,245.0,NaN,It has something to do with ecology.,worker,tunic.library.frontdesk,tunic.library.frontdesk.worker.flag,None,None,None,13-22
1,21050410434308160,978,1735199,navigate_click,undefined,20,NaN,586.732639,227.529868,762.0,155.0,NaN,None,tomicrofiche,tunic.library.frontdesk,None,None,None,None,13-22
2,21050410434308160,985,1754449,object_click,basic,20,NaN,32.837203,-229.000000,480.0,559.0,NaN,None,reader_flag.paper1.next,tunic.library.microfiche,None,None,None,None,13-22
3,21050410434308160,986,1758732,object_hover,undefined,20,NaN,NaN,NaN,NaN,NaN,10699.0,None,reader_flag.paper0.next,tunic.library.microfiche,None,None,None,None,13-22
4,21050410434308160,987,1759332,object_hover,undefined,20,NaN,NaN,NaN,NaN,NaN,566.0,None,reader_flag.paper2.bingo,tunic.library.microfiche,None,None,None,None,13-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634645,22100221145014656,1570,5431389,object_click,basic,21,NaN,-257.274472,45.755580,107.0,286.0,NaN,None,journals_flag.hub.topics,tunic.historicalsociety.stacks,None,None,None,None,13-22
2634646,22100221145014656,1584,5445520,object_hover,undefined,21,NaN,NaN,NaN,NaN,NaN,404.0,None,journals_flag.pic_1.next,tunic.historicalsociety.stacks,None,None,None,None,13-22
2634647,22100221145014656,1586,5447903,object_click,basic,21,NaN,447.725528,-72.244420,812.0,404.0,NaN,None,journals_flag.pic_2.next,tunic.historicalsociety.stacks,None,None,None,None,13-22
2634648,22100221145014656,1590,5470870,navigate_click,undefined,22,NaN,10.725528,-282.244420,375.0,614.0,NaN,None,toentry,tunic.historicalsociety.stacks,None,None,None,None,13-22


In [49]:
df_train_on = df_train.select(train_cols_interest)
df_teste_on = df_teste.select(teste_cols_interest)

In [50]:
df_train_on.printSchema()
df_teste_on.printSchema()

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- fqid: string (nullable = true)
 |-- room_fqid: string (nullable = true)
 |-- level_group: string (nullable = true)

root
 |-- session_id: long (nullable = true)
 |-- index: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- room_coor_x: double (nullable = true)
 |-- room_coor_y: double (nullable = true)
 |-- screen_coor_x: double (nullable = true)
 |-- screen_coor_y: double (nullable = true)
 |-- hover_duration: double (nullable =